In [0]:
data = [(1,"Aditya","Fevicol","Asia",3,"2020-01-04"),(2,"Avishkar","Tape","Asia",2,"2021-12-04"),(3,"Tanmay","Tape","Asia",1,"2023-1-10"),(4,"Rajesh","Fevicol","USA",4,"2022-02-04"),(5,"Sergio","Tape","Spain",5,"2022-01-04"),(6,"Rajesh","Fevicol","USA",6,"2022-01-04"),(7,"Rashford","Tape","England",7,"2022-01-04"),(8,"Rajesh","Fevicol","USA",8,"2022-01-04"),(9,"Ramesh","Tape","China",9,"2022-05-04"),(10,"Kim","Fevicol","USA",10,"2022-04-04"),(11,"Ronaldo","Tape","Portugal",11,"2022-03-04"),(12,"Sanket","Araldite","USA",12,"2025-12-10"),(13,"Pushkar","Araldite","China",5,"2025-12-10"),(14,"Akash","Tape","Asia",13,"2024-09-10")]

column = ["id","name","product","region","quantity","date"]

df = spark.createDataFrame(data, column)

In [0]:
df.display()

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("CustomerSales")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql import functions as F 

In [0]:
df_group = df.groupBy("region","product").agg(sum("quantity").alias("total_quantity")).show()

In [0]:
dbutils.widgets.dropdown("Region","Asia",["Asia","USA","China","Spain","England","Portugal"])
region_dropdown = dbutils.widgets.get("Region")
df_dropdown = df.filter(df.region == region_dropdown).select("name","product","quantity").show()

In [0]:
dbutils.widgets.multiselect("Product","Fevicol",["Fevicol","Tape","Araldite"])
product_multiselect = dbutils.widgets.get("Product")
product_list = product_multiselect.split(",")
df_multiselect = df.filter(F.col("product").isin(product_list)).select("name","product","quantity","region")\
                   .orderBy(desc("quantity")).show()




In [0]:
df_month =  df.withColumn("Month",date_format("date","MMM yyyy")).show()

In [0]:
window_spec = Window.partitionBy("region").orderBy(desc("quantity"))
df_mth = df.withColumn("rank",row_number().over(window_spec))
df_filter = df_mth.where((df_mth.rank <= 3) & (df_mth.region == "USA"))
df_filter.show()


In [0]:
spark.sql("DROP TABLE IF EXISTS CustomerSales")
df.write.mode("overwrite").partitionBy("region").format("delta").saveAsTable("CustomerSales")

In [0]:
%sql
DESCRIBE HISTORY CustomerSales;

In [0]:
df_duplicate = df.dropDuplicates(["name", "region", "product"])
df_dup = df.dropDuplicates()
df_duplicate.count()


In [0]:
%sql
optimize Customersales zorder by (date)

In [0]:
%sql
select * from CustomerSales order by id;

In [0]:
dbutils.notebook.exit("SUCCESS")